In [3]:
import pickle


agents_to_paths_file = {
    "HeavyVehicle": "samples/map_1_edges_with_attributes_HeavyVehicleModelv2.pkl",
    "LightVehicle": "samples/map_1_edges_with_attributes_LightVehicleModelv2.pkl",
    "HeavyHuman": "samples/map_1_edges_with_attributes_HeavyHumanModelv2.pkl",
    "LightHuman": "samples/map_1_edges_with_attributes_LightHumanModelv2.pkl",
}
heavy_human_file = agents_to_paths_file["HeavyHuman"]
print(heavy_human_file)

light_human_file = agents_to_paths_file["LightHuman"]
print(light_human_file)

with open(heavy_human_file, "rb") as f:
    heavy_human = pickle.load(f)

with open(light_human_file, "rb") as f:
    light_human = pickle.load(f)


heavy_vehicle_file = agents_to_paths_file["HeavyVehicle"]
print(heavy_vehicle_file)
light_vehicle_file = agents_to_paths_file["LightVehicle"]
print(light_vehicle_file)

with open(heavy_vehicle_file, "rb") as f:
    heavy_vehicle = pickle.load(f)

with open(light_vehicle_file, "rb") as f:
    light_vehicle = pickle.load(f)



list_of_paths_dict = {}

list_of_paths_dict["HeavyHuman"] =[]
for edge in heavy_human:
    path = edge['path']
    list_of_paths_dict["HeavyHuman"].append(path)

list_of_paths_dict["LightHuman"] =[]
for edge in light_human:
    path = edge['path']
    list_of_paths_dict["LightHuman"].append(path)


list_of_paths_dict["HeavyVehicle"] =[]
list_of_paths_dict["LightVehicle"] =[]
for edge in heavy_vehicle:
    path = edge['path']
    list_of_paths_dict["HeavyVehicle"].append(path)

for edge in light_vehicle:
    path = edge['path']
    list_of_paths_dict["LightVehicle"].append(path)



samples/map_1_edges_with_attributes_HeavyHumanModelv2.pkl
samples/map_1_edges_with_attributes_LightHumanModelv2.pkl
samples/map_1_edges_with_attributes_HeavyVehicleModelv2.pkl
samples/map_1_edges_with_attributes_LightVehicleModelv2.pkl


In [5]:
len(list_of_paths_dict["HeavyVehicle"])

110

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d
from PIL import Image

def generate_voronoi_seeds_overlay(bounds, out_png="static/mission_all_paths_overlay.png", shape=(1024, 1024), tasks=None, paths=None):
    """
    Rasterize Voronoi polygons into a color-coded overlay PNG.

    Parameters
    ----------
    vor : scipy.spatial.Voronoi
        Voronoi object containing vertices and regions.
    bounds : tuple
        (minx, miny, maxx, maxy) of your AOI (in lon/lat).
    attr_values : list or np.ndarray, optional
        Per-point attribute (e.g., elevation mean or landcover id) for coloring.
        Must match number of input points (len(vor.points)).
    out_png : str
        Path to save PNG.
    shape : tuple
        (H, W) output raster size.
    """
    H, W = shape
    minx, miny, maxx, maxy = bounds

    print(f"⏳ Generating Voronoi overlay ({W}×{H})...")

    fig, ax = plt.subplots(figsize=(W/300, H/300))

    # voronoi_plot_2d(vor, ax=ax, show_vertices=False, show_points=True, point_colors="red", point_size=0.5, line_colors="black", line_width=0.7, line_alpha=0.0)

    # points = vor.points
    # # ax.plot(points[:, 0], points[:, 1], 'o', color='red', markersize=0.25)
    # ax.scatter(points[:, 0], points[:, 1], s=1, color="red", edgecolor="white", linewidth=0.3, zorder=5)

    # Plot tasks if provided
    if tasks is not None and len(tasks) > 0:
        ax.scatter(tasks[:, 0], tasks[:, 1], s=80, c="black", marker="x", label="Tasks")

    # Plot paths if provided
    if paths is not None:
        for i, path in enumerate(paths):
            path = np.array(path)
            if len(path) > 0:
                ax.plot(path[:, 0], path[:, 1], '-', lw=2, label=f"Path {i+1}")




    ax.set_xlim(0, W)
    ax.set_ylim(H, 0)  # invert Y to match image orientation
    ax.set_xticks([])
    ax.set_yticks([])
    plt.tight_layout(pad=0)
    plt.savefig(out_png, dpi=300, transparent=True)
    plt.close(fig)
    print(f"✅ Saved Voronoi overlay plot: {out_png}")

    # Image.fromarray(img).save(out_png)
    # print(f"✅ Saved {out_png}")

    # Write .wld file for Cesium alignment
    pixel_width = (maxx - minx) / W
    pixel_height = (miny - maxy) / H
    with open(out_png.replace(".png", ".wld"), "w") as f:
        f.write(f"{pixel_width}\n0.0\n0.0\n{pixel_height}\n{minx}\n{maxy}\n")

    print(f"🗺️  Saved world file: {out_png.replace('.png', '.wld')}")

In [6]:
with open("./samples/map_1_data_with_voronoi.pkl", "rb") as f:
    map_1_data = pickle.load(f)
print(map_1_data.keys())
dem_array1 = map_1_data['dem_array']
landcover_array1 = map_1_data['landcover_array']
sat_image_array1 = map_1_data['sat_image_array']
# all_paths_1 = map_1_data['all_paths']
vor = map_1_data['voronoi']
bounds = map_1_data['bounds']

dict_keys(['dem_array', 'landcover_array', 'sat_image_array', 'voronoi', 'transform', 'crs', 'bounds'])


In [11]:
import numpy as np
# Example usage
tasks = np.array([[100, 1900], 
                  [3500, 750], 
                  [1600, 1000], #Vehicle Depot
                  [300, 400], 
                  [3200, 800],
                  [3100, 1500],
                  [500, 1100],
                  [1300, 1750],
                  [2450, 1800], 
                  [3000, 300],
                  [3500, 1200] #Human Depot
                  ])

In [14]:
for agent in agents_to_paths_file:
    paths = list_of_paths_dict[agent]

    generate_voronoi_seeds_overlay(bounds, out_png=f"static/{agent}mission_all_paths_overlay.png", shape=dem_array1.shape, tasks=None, paths=paths)

⏳ Generating Voronoi overlay (3715×2051)...
✅ Saved Voronoi overlay plot: static/HeavyVehiclemission_all_paths_overlay.png
🗺️  Saved world file: static/HeavyVehiclemission_all_paths_overlay.wld
⏳ Generating Voronoi overlay (3715×2051)...
✅ Saved Voronoi overlay plot: static/LightVehiclemission_all_paths_overlay.png
🗺️  Saved world file: static/LightVehiclemission_all_paths_overlay.wld
⏳ Generating Voronoi overlay (3715×2051)...
✅ Saved Voronoi overlay plot: static/HeavyHumanmission_all_paths_overlay.png
🗺️  Saved world file: static/HeavyHumanmission_all_paths_overlay.wld
⏳ Generating Voronoi overlay (3715×2051)...
✅ Saved Voronoi overlay plot: static/LightHumanmission_all_paths_overlay.png
🗺️  Saved world file: static/LightHumanmission_all_paths_overlay.wld


In [17]:
def read_worldfile(path):
    with open(path) as f:
        vals = list(map(float, f.read().split()))
    A, D, B, E, C, F = vals  # typical .wld order
    return A, D, B, E, C, F

A, D, B, E, C, F = read_worldfile("static/voronoi_overlay.wld")

def to_geo_from_worldfile(x, y):
    lon = A * x + B * y + C
    lat = D * x + E * y + F
    return [lon, lat]

geo_points = [to_geo_from_worldfile(x, y) for x, y in tasks]


In [18]:
geo_points

[[-116.69014300133762, 34.242955807461634],
 [-115.7741717137646, 34.55212071346959],
 [-116.286038021526, 34.48491095129395],
 [-116.63626233736274, 34.64621438051549],
 [-115.85499270972693, 34.53867876103446],
 [-115.88193304171438, 34.35049142694266],
 [-116.58238167338786, 34.45802704642369],
 [-116.36685901748832, 34.283281664767024],
 [-116.05704519963274, 34.269839712331894],
 [-115.90887337370181, 34.67309828538574],
 [-115.7741717137646, 34.431143141553434]]

## COA 1

In [144]:

ag_waypoints = {}
ag_waypoints["HeavyVehicle"] = [[1600, 1000], [100, 1900], [300, 400], [1300, 1750], [2450, 1800], [1600, 1000]]
ag_waypoints["LightVehicle"] = [[1600, 1000], [500, 1100], [1600, 1000]]
ag_waypoints["HeavyHuman"] = [[3500, 1200], [3200, 800], [3100, 1500], [3500, 1200]]
ag_waypoints["LightHuman"] = [[3500, 1200], [3500, 750], [3000, 300], [3500, 1200]]



def generate_agent_dict_for_czml(ag_waypoints, list_of_paths_dict):
# # covert to geo points
# for key in ag_waypoints.keys():
#     ag_waypoints[key] = [to_geo_from_worldfile(x, y) for x, y in ag_waypoints[key]]


    agent_dicts = []

    for key in ag_waypoints.keys():
        dic = {}
        waypoints = []
        for i in range(1, len(ag_waypoints[key])):
            start = ag_waypoints[key][i-1]
            end = ag_waypoints[key][i]
            paths = list_of_paths_dict[key]
            for path in paths:
                if list(path[0]) == start and list(path[-1]) == end:
                    break
            for point in path:
                waypoints.append(point)
        dic["waypoints"] = waypoints
        if key == "LightVehicle":
            dic["agent_type"] = "LightVehicle"
            dic["speed"] = 5
            dic["symbol"] = "Symbols/LightArmoredRecon.png"
        elif key == "HeavyVehicle":
            dic["agent_type"] = "HeavyVehicle"
            dic["speed"] = 3
            dic["symbol"] = "Symbols/WheeledArmored.png"
        elif key == "LightHuman":
            dic["agent_type"] = "LightHuman"
            dic["speed"] = 1.5
            dic["symbol"] = "Symbols/LightInfantry.png"
        elif key == "HeavyHuman":
            dic["agent_type"] = "HeavyHuman"
            dic["speed"] = 1.5
            dic["symbol"] = "Symbols/ArmoredInfantry.png"
        
        agent_dicts.append(dic)
    return agent_dicts
agent_dicts = generate_agent_dict_for_czml(ag_waypoints, list_of_paths_dict)

In [29]:
ag_waypoints

{'HeavyVehicle': [[1600, 1000],
  [100, 1900],
  [300, 400],
  [1300, 1750],
  [2450, 1800],
  [1600, 1000]],
 'LightVehicle': [[1600, 1000], [500, 1100], [1600, 1000]],
 'HeavyHuman': [[3500, 1200], [3200, 800], [3100, 1500], [3500, 1200]],
 'LightHuman': [[3500, 1200], [3500, 750], [3000, 300], [3500, 1200]]}

In [ ]:
import json
from datetime import datetime, timedelta
import numpy as np

def make_czml(agent_dicts, elevation_map, worldfile=None, start_time=None, out_file="agents.czml"):
    """
    Generate CZML animation file for agents with waypoints and speed.
    """

    color_map = {
    "HeavyVehicle": [255, 100, 100, 200],
    "LightVehicle": [255, 255, 100, 200],
    "HeavyHuman": [100, 200, 255, 200],
    "LightHuman": [100, 255, 150, 200]
    }


    from datetime import timedelta
    import numpy as np

    # Timeline setup
    if start_time is None:
        start_time = datetime.utcnow()

    # --- compute maximum travel time among all agents ---
    max_duration = 0.0
    for agent in agent_dicts:
        wps = np.array(agent["waypoints"])
        if len(wps) < 2:
            continue
        diffs = np.linalg.norm(np.diff(wps, axis=0), axis=1)
        path_len = float(np.sum(diffs))
        duration = path_len / agent["speed"]  # seconds
        max_duration = max(max_duration, duration)

    end_time = start_time + timedelta(seconds=max_duration)




    czml = [{
        "id": "document",
        "name": "Agent Animation",
        "version": "1.0",
        "clock": {
            "interval": f"{start_time.isoformat()}Z/{(start_time + timedelta(minutes=5000)).isoformat()}Z",
            # "interval": f"{start_time.isoformat()}Z/{end_time.isoformat()}Z",
            "currentTime": f"{start_time.isoformat()}Z",
            "multiplier": 1,
            "range": "LOOP_STOP"
        }
    }]

    for idx, agent in enumerate(agent_dicts):
        wps = agent["waypoints"]
        speed = agent.get("speed", 1.0)
        symbol = agent.get("symbol", None)

        # build time-stamped positions
        pos_list = []
        t = start_time
        for i in range(1, len(wps)):
            x1, y1 = wps[i-1]
            x2, y2 = wps[i]
            lon1, lat1 = to_geo_from_worldfile(x1, y1)
            lon2, lat2 = to_geo_from_worldfile(x2, y2)

            # print(lon1, lat1, lon2, lat2)

            # approximate distance in meters (flat-earth)
            dx = (lon2 - lon1) * 111320 * np.cos(np.radians(lat1))
            dy = (lat2 - lat1) * 110540
            dist = np.sqrt(dx**2 + dy**2)
            dt = dist / speed
            # print(f"Agent {x1,y1} to {x2,y2}: {dist:.1f}m in {dt:.1f}s at {speed}m/s")
            h1 = float(elevation_map[int(y1), int(x1)])
            h2 = float(elevation_map[int(y2), int(x2)])
            # add points (time offset, lon, lat, alt)
            pos_list.extend([
                (t - start_time).total_seconds(), lon1, lat1, h1,
                (t - start_time).total_seconds() + dt, lon2, lat2, h2
            ])
            t += timedelta(seconds=dt)

            

        czml.append({
            "id": f"agent_{idx}",
            "name": f"Agent {idx}",
            "availability": f"{start_time.isoformat()}Z/{t.isoformat()}Z",
            "position": {
                "epoch": f"{start_time.isoformat()}Z",
                "cartographicDegrees": pos_list,
                "interpolationAlgorithm": "LINEAR",
                "interpolationDegree": 1
            },
            "billboard": {
                "image": symbol,
                "scale": 0.14,
                "verticalOrigin": "BOTTOM"
            },
            "path": {
                "material": {
                    "polylineOutline": {
                        "color": {"rgba": [0, 255, 255, 180]},
                        "outlineColor": {"rgba": [0, 0, 0, 255]},
                        "outlineWidth": 1
                    }
                },
                "width": 2,
                "leadTime": 0,
                "trailTime": 60
            }
        })


        # --- Full static path line ---
        static_positions = [p for i, p in enumerate(pos_list) if i % 4 != 0]  # strip time values
        rgba = color_map.get(key, [255, 255, 255, 180])  # fallback white

        czml.append({
            "id": f"path_{idx}",
            "name": f"Route {idx}",
            "polyline": {
                "positions": {
                    "cartographicDegrees": static_positions,
                },
                "width": 5.5,
                 "clampToGround": True,
                "material": {"solidColor": {"color": {"rgba": rgba}}}
            }
        })

    with open(out_file, "w") as f:
        json.dump(czml, f, indent=2)

    print(f"✅ Saved CZML animation → {out_file}")


In [68]:
make_czml(agent_dicts, dem_array1, worldfile=[A, D, B, E, C, F], start_time=None, out_file="static/agents_coa_1.czml")

✅ Saved CZML animation → static/agents_coa_1.czml


In [117]:
import json

def make_task_points_czml(task_locations, depot_indices=None, worldfile=None, out_file="task_points.czml"):
    """
    Generate CZML for task points with depot points highlighted.

    Parameters
    ----------
    task_locations : list of [x, y]
        Pixel coordinates (or lon/lat if no worldfile given).
    depot_indices : list of int, optional
        Indices (0-based) of depot locations to color differently.
    worldfile : list or tuple of 6 floats, optional
        [A, D, B, E, C, F] worldfile transform for pixel→geo conversion.
    out_file : str
        Path to save the CZML file.
    """

    def pixel_to_geo(x, y):
        if not worldfile:
            return x, y
        A, D, B, E, C, F = worldfile
        lon = A * x + B * y + C
        lat = D * x + E * y + F
        return lon, lat

    if depot_indices is None:
        depot_indices = []

    czml = [{
        "id": "document",
        "name": "Task Points",
        "version": "1.0"
    }]
    depot_count = 0
    for idx, (x, y) in enumerate(task_locations, start=1):
        lon, lat = pixel_to_geo(x, y)
        is_depot = (idx - 1) in depot_indices

        # 🎨 color scheme
        if is_depot:
            color = [0, 255, 255, 255]        # Cyan depot
            outline = [0, 100, 255, 255]
            pixel_size = 22
            depot_count += 1
        else:
            color = [255, 255, 255, 255]      # White task
            outline = [255, 0, 0, 255]
            pixel_size = 18

        czml.append({
            "id": f"task_{idx}",
            "name": f"{'D' if is_depot else 'T'} {depot_count if is_depot else idx}",
            "position": {
                "cartographicDegrees": [lon, lat, 2],  # Slightly above ground
                "heightReference": "CLAMP_TO_GROUND"
            },
            "point": {
                "color": {"rgba": color},
                "outlineColor": {"rgba": outline},
                "outlineWidth": 3,
                "pixelSize": pixel_size,
                "heightReference": "CLAMP_TO_GROUND"
            },
            "label": {
                "text": f"{'D' if is_depot else 'T'} {depot_count if is_depot else idx}",
                "font": "14px sans-serif",
                "pixelOffset": {"cartesian2": [0, -25]},
                "fillColor": {"rgba": [255, 255, 0, 255]},
                "showBackground": True,
                "backgroundColor": {"rgba": [0, 0, 0, 150]},
                "heightReference": "CLAMP_TO_GROUND"
            }
        })

    with open(out_file, "w") as f:
        json.dump(czml, f, indent=2)

    print(f"✅ CZML saved with {len(task_locations)} points ({len(depot_indices)} depots) → {out_file}")
    # return czml


In [118]:

# Mark depot indices (0-based)
depot_indices = [2, 10]

make_task_points_czml(tasks, depot_indices=depot_indices, worldfile=[A, D, B, E, C, F], out_file="static/task_points_1.czml")

✅ CZML saved with 11 points (2 depots) → static/task_points_1.czml


## Actual Models

In [142]:
import json
from datetime import datetime, timedelta
import numpy as np

def make_czml_3d(agent_dicts, elevation_map, to_geo_from_worldfile, start_time=None, out_file="agents.czml"):
    """
    Generate CZML animation file for agents with 3D human/vehicle models and path trails.
    """

    color_map = {
        "HeavyVehicle": [255, 100, 100, 200],
        "LightVehicle": [255, 255, 100, 200],
        "HeavyHuman": [100, 200, 255, 200],
        "LightHuman": [100, 255, 150, 200]
    }

    model_map = {
        "Vehicle": "/static/GroundVehicle.glb",
        "Human": "/static/Cesium_Man.glb"
    }

    # --- Setup timeline ---
    if start_time is None:
        start_time = datetime.utcnow()

    # # Compute max duration
    # max_duration = 0.0
    # for agent in agent_dicts:
    #     wps = np.array(agent["waypoints"])
    #     if len(wps) < 2:
    #         continue
    #     diffs = np.linalg.norm(np.diff(wps, axis=0), axis=1)
    #     path_len = float(np.sum(diffs))
    #     duration = path_len / agent["speed"]
    #     max_duration = max(max_duration, duration)

    # end_time = start_time + timedelta(seconds=max_duration)

    czml = [{
        "id": "document",
        "name": "Agent Animation",
        "version": "1.0",
        "clock": {
            "interval": f"{start_time.isoformat()}Z/{(start_time + timedelta(minutes=5000)).isoformat()}Z",
            "currentTime": f"{start_time.isoformat()}Z",
            "multiplier": 1,
            "range": "LOOP_STOP"
        }
    }]

    # --- Build each agent ---
    for idx, agent in enumerate(agent_dicts):
        key = agent.get("agent_type", f"Agent{idx}")  # you can set agent["type"] = "LightVehicle", etc.
        wps = agent["waypoints"]
        speed = agent.get("speed", 1.0)

        # Choose model & color based on agent type
        if "Vehicle" in key:
            model_path = model_map["Vehicle"]
            model_scale = 30.0 # 1.2
        else:
            model_path = model_map["Human"]
            model_scale = 20.0  # 0.6

        rgba = color_map.get(key, [255, 255, 255, 180])
        pos_list = []
        t = start_time

        for i in range(1, len(wps)):
            x1, y1 = wps[i - 1]
            x2, y2 = wps[i]
            lon1, lat1 = to_geo_from_worldfile(x1, y1)
            lon2, lat2 = to_geo_from_worldfile(x2, y2)

            # Approx distance in meters
            dx = (lon2 - lon1) * 111320 * np.cos(np.radians(lat1))
            dy = (lat2 - lat1) * 110540
            dist = np.sqrt(dx ** 2 + dy ** 2)
            dt = dist / speed

            h1 = float(elevation_map[int(y1), int(x1)])
            h2 = float(elevation_map[int(y2), int(x2)])

            pos_list.extend([
                (t - start_time).total_seconds(), lon1, lat1, h1,
                (t - start_time).total_seconds() + dt, lon2, lat2, h2
            ])
            t += timedelta(seconds=dt)

        # --- Animated model agent ---
        czml.append({
            "id": f"agent_{idx}",
            "name": key,
            "availability": f"{start_time.isoformat()}Z/{t.isoformat()}Z",
            "position": {
                "epoch": f"{start_time.isoformat()}Z",
                "cartographicDegrees": pos_list,
                "interpolationAlgorithm": "LINEAR",
                "interpolationDegree": 1
            },
            "orientation": {
                "velocityReference": "#position"
            },
            "model": {
                "gltf": model_path,
                "scale": model_scale,
                "minimumPixelSize": 64,
                "runAnimations": True,
                "heightReference": "CLAMP_TO_GROUND"
            },
            "path": {
                "material": {
                    "polylineOutline": {
                        "color": {"rgba": rgba},
                        "outlineColor": {"rgba": [0, 0, 0, 255]},
                        "outlineWidth": 1
                    }
                },
                "width": 3,
                "leadTime": 0,
                "trailTime": 60,
                "clampToGround": True
            }
        })

        # --- Static route line ---
        static_positions = [p for i, p in enumerate(pos_list) if i % 4 != 0]
        czml.append({
            "id": f"path_{idx}",
            "name": f"Route {idx}",
            "polyline": {
                "positions": {"cartographicDegrees": static_positions},
                "width": 5,
                "clampToGround": True,
                "material": {"solidColor": {"color": {"rgba": rgba}}}
            }
        })

    # --- Write file ---
    with open(out_file, "w") as f:
        json.dump(czml, f, indent=2)

    print(f"✅ Saved CZML with {len(agent_dicts)} agents → {out_file}")


In [143]:
make_czml_3d(agent_dicts, dem_array1, to_geo_from_worldfile, start_time=None, out_file="static/agents_coa_3d_1.czml")

✅ Saved CZML with 4 agents → static/agents_coa_3d_1.czml


In [127]:
agent_dicts[0].keys()

dict_keys(['waypoints', 'speed', 'symbol'])

In [145]:
ag_waypoints_2 = {}
ag_waypoints_2["HeavyVehicle"] = [[1600, 1000], [100, 1900], [2450, 1800], [1600, 1000]]
ag_waypoints_2["LightVehicle"] = [[1600, 1000], [300, 400], [500, 1100], [1300, 1750], [1600, 1000]]
ag_waypoints_2["HeavyHuman"] = [[3500, 1200], [3500, 750], [3500, 1200]]
ag_waypoints_2["LightHuman"] = [[3500, 1200], [3200, 800], [3100, 1500], [3000, 300], [3500, 1200]]

agent_dicts_2 = generate_agent_dict_for_czml(ag_waypoints_2, list_of_paths_dict)

In [146]:

# Mark depot indices (0-based)
depot_indices = [2, 10]

make_task_points_czml(tasks, depot_indices=depot_indices, worldfile=[A, D, B, E, C, F], out_file="static/task_points_2.czml")

✅ CZML saved with 11 points (2 depots) → static/task_points_2.czml


In [147]:
make_czml_3d(agent_dicts_2, dem_array1, to_geo_from_worldfile, start_time=None, out_file="static/agents_coa_3d_2.czml")

✅ Saved CZML with 4 agents → static/agents_coa_3d_2.czml
